In [1]:
import requests
import pandas as pd
from datetime import datetime, date

API_URL = "https://api.alternative.me/fng/?limit=0&format=json&date_format=world"
OUTPUT_CSV = "/Users/apple/Desktop/time-series/informer-validation/data/fear_greed_index.csv"


def fetch_fng_history() -> pd.DataFrame:
    """Fetch full F&G history and return as a DataFrame."""
    resp = requests.get(API_URL, timeout=10)
    resp.raise_for_status()
    data = resp.json().get("data", [])
    
    # Build DataFrame
    df = pd.DataFrame(data)
    
    # Parse the timestamp column which contains date strings (e.g., '06-08-2025')
    df["datetime"] = pd.to_datetime(df["timestamp"], format="%d-%m-%Y").dt.date
    df["datetime"] = pd.to_datetime(df["datetime"])
    
    # Convert numeric fields
    df["value"] = pd.to_numeric(df["value"], errors='coerce')
    df["value_classification"] = df["value_classification"].astype(str)
    
    # Keep only relevant columns (and sort ascending by date)
    df = df[["datetime", "value", "value_classification"]].sort_values("datetime").reset_index(drop=True)
    return df



print("Fetching Fear & Greed Index history…")
df = fetch_fng_history()
print(f"Total records fetched: {len(df)}")


df.to_csv(OUTPUT_CSV, index=False)

df

Fetching Fear & Greed Index history…
Total records fetched: 2740


,datetime,value,value_classification
0,2018-02-01,30,Fear
1,2018-02-02,15,Extreme Fear
2,2018-02-03,40,Fear
3,2018-02-04,24,Extreme Fear
4,2018-02-05,11,Extreme Fear
...,...,...,...
2735,2025-08-02,55,Greed
2736,2025-08-03,53,Neutral
2737,2025-08-04,64,Greed
2738,2025-08-05,60,Greed


In [2]:
df = pd.read_csv(OUTPUT_CSV, parse_dates=["datetime"])
df

,datetime,value,value_classification
0,2018-02-01,30,Fear
1,2018-02-02,15,Extreme Fear
2,2018-02-03,40,Fear
3,2018-02-04,24,Extreme Fear
4,2018-02-05,11,Extreme Fear
...,...,...,...
2735,2025-08-02,55,Greed
2736,2025-08-03,53,Neutral
2737,2025-08-04,64,Greed
2738,2025-08-05,60,Greed
